## Compress the Base LLM using LLM Compressor:
In this step, a Large Language Model(which we refere to as the base model for this use case) is compressed to reduce its memory footprint and improve inference efficiency without significantly impacting accuracy.

**Goal**: Reduce model size (e.g., FP16 → INT8/INT4) while retaining performance.

**Key Actions**:

- Load the base model.

- Measure its size and memory usage.

- Use a calibration dataset (e.g., WikiText, UltraChat) to collect activation statistics.

- Apply a quantization recipe (e.g., SmoothQuant + GPTQ modifier).

- Save the compressed model and verify size reduction.

**Outcome**:

- Compressed model saved on disk.

- Model size reduced, typically by 50% (depending on quantization scheme).

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from llmcompressor.modifiers.quantization import QuantizationModifier, GPTQModifier
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor import oneshot
from datasets import load_dataset
import torch
import os
from utils import model_size_gb, tokenize_for_calibration

/opt/app-root/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-10 11:27:43,384	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 12-10 11:27:43 [__init__.py:216] Automatically detected platform cuda.


In [2]:
# check available device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Loading Base Model
You can use the model of your choice by modifying the "model_name" variable.
While loading the model using **from_pretrained** using transformers' **AutoModelForCausalLM** class, we specify the data type using the **torch_dtype** parameter and set it to **auto** so the model is loaded in the data type specified in its config.
Otherwise, PyTorch loads the weights in **full precision (fp32)**.

In [4]:
# set up variables
model_name = "RedHatAI/Llama-3.1-8B-Instruct"
base_model_path = "./base_model"
compressed_model_path = "Llama-3.1-8B-Instruct-int8-dynamic"

In [5]:
# loading model and tokenizer from huggingfaceabs
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype="auto",
                                             device_map="auto")
model.config.dtype = "bfloat16"
# saving model and tokenizer
model.save_pretrained(base_model_path)
tokenizer.save_pretrained(base_model_path)

print("Base model saved at:", base_model_path)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 127.76it/s]


Base model saved at: ./base_model


In [6]:
# check model size   
# !du -sh {base_model_path}
model_size = model_size_gb(model)
print(f"The size of the base model is: {model_size:.4f}GB")

The size of the base model is: 14.9575GB


## Data Aware Weight+Activation Quantization

- Use a calibration dataset to avoid data distribution drift after quantization
- Scheme for quantization is "W8A8"; convert both weights and activations to INT8 (can be W4A4 as well)
- Activations are quantized on the fly during inference. (dynamic activation)

### Things to keep in mind for data aware quantization
1. **Choice of Calibration Dataset:** GPTQ quantization estimates activation ranges from calibration data. If the calibration data is very different from what the model will see in production, these ranges may be inaccurate, leading to higher quantization errors and degraded model outputs.

   For production, use a dataset that closely resembles the expected domain(e.g finance, medicine etc), task(Q/A,    summarization etc), and style of your inputs to ensure quantization preserves quality.

    For the sake of this demo, we can use a small, general-purpose dataset for faster processing. Specifically, we use the `wikitext-2-raw-v1` version of the WikiText dataset which is the smaller version.

2. **Number of Calibration Samples Used for Quantization**

     More samples give better and stable statistics on the range and distribution of activations, which reduces quantization noise. Small calibration sets, on the other hand, are quicker but noisier.
    
    For the sake of this demo, we use a small number of samples (e.g., 16–512) is enough to show the process.
    
    For production, use a larger sample set (hundreds to thousands) to stabilize ranges and minimize error.

4. **Sequence Length**

    Longer input sequences generate larger activations because each token’s representation depends on all previous tokens and layers. These bigger values can exceed the quantization range (e.g., -128 to 127 for 8-bit quantization), causing rounding errors or clipping, which reduces accuracy.
    
    For this demo, shorter sequences are sufficient to illustrate quantization.
    
    For production, use sequences that reflect maximum expected lengths in your application to prevent errors.



### Preparing Calibration Dataset

In [11]:
# Define the dataset to use for calibration
dataset_id = "wikitext"  

# Specify the configuration / version of the dataset
config = "wikitext-2-raw-v1"  # Small version (~2M tokens), raw text format

# Set the number of calibration samples based on available device
# - On GPU: use more samples to get more accurate activation statistics
# - On CPU: reduce samples to prevent memory issues and keep demo fast
num_calibration_samples = 512 if device == "cuda" else 16  

# Set the maximum sequence length for calibration
max_sequence_length = 1024 if device == "cuda" else 16  

# Load the dataset using Hugging Face Datasets API
# This downloads train split of the dataset
ds = load_dataset(dataset_id, config, split="train")  
# Shuffle and grab only the number of samples we need
ds = ds.shuffle(seed=42).select(range(num_calibration_samples))

In [12]:
# inspect the dataset
print(f"columns in the {dataset_id}: {ds.column_names}\n")
print(ds[0])

columns in the wikitext: ['text']

{'text': ' Continuous , short @-@ arc , high pressure xenon arc lamps have a color temperature closely approximating noon sunlight and are used in solar simulators . That is , the chromaticity of these lamps closely approximates a heated black body radiator that has a temperature close to that observed from the Sun . After they were first introduced during the 1940s , these lamps began replacing the shorter @-@ lived carbon arc lamps in movie projectors . They are employed in typical 35mm , IMAX and the new digital projectors film projection systems , automotive HID headlights , high @-@ end " tactical " flashlights and other specialized uses . These arc lamps are an excellent source of short wavelength ultraviolet radiation and they have intense emissions in the near infrared , which is used in some night vision systems . \n'}


**Datset inspection shows the we need to extract column ```text``` and pass it as input to the model.**

### When to Use a Custom Template for Calibration

Use a **custom template** when you want the calibration text to closely mimic the input format your model will see in production.  

For example, if your model is **instruction-following** or **chat-based**, providing the template the model was originally trained on or the template that will be used during inference ensures that the activation statistics collected during calibration reflect realistic usage patterns. 

This can improve the accuracy of quantization and compression.

If your model can handle raw text and doesn’t require a specific format, you can rely on the default template instead.




In [ ]:
# to get activations for the calibration dataset, we need to:
# 1. extract the samples from the dataset 
# 2. tokenize samples in the dataset
input_column = "text"

# Call tokenize_for_calibration using dataset.map
tokenized_dataset = ds.map(
    lambda batch: tokenize_for_calibration(
        examples=batch,                   # batch from Hugging Face dataset
        input_column=input_column,        # the column containing text to calibrate
        tokenizer=tokenizer,              # your Hugging Face tokenizer
        max_length=max_sequence_length,   # maximum sequence length
        model_type="chat",                # use chat template if no custom template
        custom_template=None              # optional, provide a dict if you want a custom template
    ),
    batched=True
)

In [14]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 512
})

### Quantizing/Compressing Base Model to INT8

**SmoothQuant** SmoothQuant operates on the activations (outputs of intermediate layers that become inputs to the next layer) produced by the base model. These activations can sometimes have extreme values (outliers). SmoothQuant scales the activations to reduce these outliers so that most values fall within a reasonable range, e.g., [-4, 4].

To ensure that the overall layer output remains unchanged (Y = W * A), SmoothQuant also scales the corresponding weights by multiplying them with the same factor.

Activations are scaled as:
$A^*=A/s$

Weights are scaled as:
$W^*=W∗s$

This way, the layer output remains approximately the same, but the activations are now suitable for stable low-bit quantization.

**GPTQModifier** GPTQ takes the smoothed activations and weights produced by SmoothQuant and computes a quantization scale for each weight matrix. This scale determines how weights will be mapped into low-bit integers (e.g., int8).

GPTQ then:

1. Quantizes the weights using these scales

    $Wquant=round(W/s)$

2. Computes the model outputs using:

    full-precision weights → Y

   
    quantized weights → Yquant

3. Adjusts the quantization error so that

    $Yquant≈Y$


In [ ]:
# Define the quantization scheme
scheme = "W8A8"  # W8A8 means 8-bit weights and 8-bit activations

# Strength for SmoothQuant smoothing
# This controls how much the activation values are smoothed to reduce outliers
smoothing_strength = 0.8

# Create SmoothQuant modifier
# - smooths activations before quantization to improve stability and reduce degradation
smooth_quant = SmoothQuantModifier(smoothing_strength=smoothing_strength)

# Create GPTQ modifier
# - targets="Linear" quantizes only Linear layers (e.g., feedforward layers)
# - scheme=scheme uses the W8A8 quantization scheme
# - ignore=["lm_head"] preserves the LM head to avoid generation quality loss
quantizer = GPTQModifier(targets="Linear", scheme=scheme, ignore=["lm_head"])

# Combine the modifiers into a recipe list
# The order matters: first apply SmoothQuant, then GPTQ
recipe = [
    smooth_quant,
    quantizer
]

# Perform quantization
oneshot(
    model=model_name,                 # Model to quantize
    dataset=ds,                       # Calibration dataset, used for both SmoothQuant & GPTQ
    recipe=recipe,                    # List of quantization modifiers to apply
    output_dir=compressed_model_path, # Directory to save the quantized model
    max_seq_length=2048,              # Maximum sequence length for calibration
    num_calibration_samples=512       # Number of samples used for calibration
)

Tokenizing: 100%|██████████| 512/512 [00:00<00:00, 1279.24 examples/s]

2025-12-10T11:33:12.940989+0000 | reset | WARNING - Exception during finalizing modifier: Failed to compress 1 modules
2025-12-10T11:33:12.945893+0000 | reset | INFO - Compression lifecycle reset
2025-12-10T11:33:12.948221+0000 | from_modifiers | INFO - Creating recipe from modifiers
2025-12-10T11:33:12.948976+0000 | _infer_mappings_from_model | INFO - No SmoothQuantModifier.mappings provided, inferring from model...


2025-12-10T11:33:13.704507+0000 | initialize | INFO - Compression lifecycle initialized for 2 modifiers
2025-12-10T11:33:13.705516+0000 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `SmoothQuantModifier`


(1/33): Calibrating: 100%|██████████| 512/512 [00:04<00:00, 109.09it/s]

2025-12-10T11:33:19.573547+0000 | _apply_smoothing | INFO - Smoothing with model.layers.0.input_layernorm
2025-12-10T11:33:19.577368+0000 | _apply_smoothing | INFO - Smoothing with model.layers.0.post_attention_layernorm


...

2025-12-10T11:57:53.079379+0000 | compress | METRIC - time 1.44s
2025-12-10T11:57:53.080903+0000 | compress | METRIC - error 41.56
2025-12-10T11:57:53.081548+0000 | compress | METRIC - GPU 0 | usage: 11.69% | total memory: 48 GB
2025-12-10T11:57:53.082136+0000 | compress | METRIC - Compressed module size: 33.56672 MB
2025-12-10T11:57:53.082823+0000 | compress_modules | INFO - Quantizing model.layers.31.self_attn.k_proj using 512 samples
2025-12-10T11:57:54.476772+0000 | compress | METRIC - time 1.39s
2025-12-10T11:57:54.477676+0000 | compress | METRIC - error 28.77
2025-12-10T11:57:54.478621+0000 | compress | METRIC - GPU 0 | usage: 11.69% | total memory: 48 GB
2025-12-10T11:57:54.479093+0000 | compress | METRIC - Compressed module size: 8.39168 MB
2025-12-10T11:57:54.479782+0000 | compress_modules | INFO - Quantizing model.layers.31.self_attn.v_proj using 512 samples
2025-12-10T11:57:55.882090+0000 | compress | METRIC - time 1.40s
2025-12-10T11:57:55.883174+0000 | compress | METRIC - 

(33/33): Propagating: 100%|██████████| 512/512 [00:05<00:00, 85.60it/s]


2025-12-10T11:58:24.639425+0000 | finalize | INFO - Compression lifecycle finalized for 2 modifiers
2025-12-10T11:58:24.681928+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 423it [00:04, 100.07it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [17]:
# Load quantized model
model_quant = AutoModelForCausalLM.from_pretrained(compressed_model_path)
model_quant.config.dtype = model.config.torch_dtype
model_quant.save_pretrained(compressed_model_path)
model_size = model_size_gb(model_quant)
print(f"Model size (GB): {model_size}")


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 16.16it/s]


Model size (GB): 8.460090637207031


### Observation
After quantizing the model, the size has clearly reduced from 14.9GB to 8GB. Now that we have reduced the model size, the next step is to evaluate this compressed model to make sure the accuracy has retained after compression.


**ALTERNATIVELY**, llm-compressor also supports FP8 quantization. This conversion foes not require any calibration dataset. Uncomment the below cell to quantize the model to FP8.

In [ ]:
# recipe = QuantizationModifier(
#   targets="Linear", scheme="FP8_DYNAMIC", ignore=["lm_head"])

# oneshot(model=model_name, recipe=recipe, output_dir=compressed_model_path)

# # Load quantized model
# model_quant = AutoModelForCausalLM.from_pretrained(compressed_model_path)
# model_quant.config.dtype = model.config.torch_dtype
# model_quant.save_pretrained(compressed_model_path)
# model_size = model_size_gb(model_quant)
# print(f"Model size (GB): {model_size}")
